In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
from linearmodels.panel import PanelOLS


In [2]:
df = pd.read_excel("UYdata.xlsx")

# Data Cleaning

In [3]:
# remove rows with attribute missing
df = df.loc[df.att_missing == 0]

In [4]:
# fill missing distance from sub district with google distance
df["subdistricthqdist_c"].fillna(df["google_km"], inplace=True)

In [5]:
# the total lpg refills is reported for only those households who could be 
# matched with OMC sales records (N = 2729). Therefore drop all those obvs 
# which could not be matched
df =  df.drop(
    df['totrefills_omc_b2'].index[df['totrefills_omc_b2'].apply(np.isnan)]
    )

In [6]:
# taing a subset of the dataframe (columns which are needed)
reg_data = df[["villagecode_str",   # for clustering
                "hhnum_b", "edu_hhhead_b", "occu_b", "age_pc_b", "pc_edu_b", 
                "hindu_b", "hh_caste_b", "assets_index",   #  HH controls
                "education_b", "healthstatus_b", "road_b", "irrigation_b",  
                "subdistricthqdist_c", #  Village controls
                # "noncompliance_village",
                "pcdecisionindex_std_b",
                "totrefills_omc_b2", "treatment012", "treatment_h", "treatment_hs",
                "totrefills_omc_e2", 
                "tehsil_e"      # fixed  effects
                   ]]  

# Table 6

In [7]:
# all controls in form of formula
control = "+ hhnum_b + edu_hhhead_b + occu_b + age_pc_b + pc_edu_b + hindu_b + hh_caste_b + assets_index + education_b + healthstatus_b + road_b + irrigation_b + subdistricthqdist_c"

Three characteristics used in as intercation:
1. HH heads education: edu_hhhead_b
2. HH wealth: assets_index
3. PC's bargaining power: pcdecisionindex_std_b

## Regressions: Impact of demand side characteristics on annual LPG refill consumption

In [8]:
# first model DD side char: Head's edu
DD_side_char = ["edu_hhhead_b", "assets_index", "pcdecisionindex_std_b"]
models = []
for char in DD_side_char:
      formula_1 = f"totrefills_omc_e2 ~ treatment012 + totrefills_omc_b2 + {char} * treatment012 " 
      formula_2 = f"totrefills_omc_e2 ~ treatment_h + treatment_hs + totrefills_omc_b2 + {char} * treatment_h + {char} * treatment_hs"
      
      
      if char == "pcdecisionindex_std_b":
            reg_data = reg_data.dropna()
            formula_1 = f"totrefills_omc_e2 ~ treatment012 + totrefills_omc_b2 + {char} + {char} * treatment012 " 
            formula_2 = f"totrefills_omc_e2 ~ treatment_h + treatment_hs + totrefills_omc_b2 + {char} + {char} * treatment_h + {char} * treatment_hs"
      
      # Non fixed effects model
      models.append( smf.ols(formula_1 + control,
                        data= reg_data
                        ).fit(cov_type='cluster', 
                              cov_kwds={'groups': reg_data["villagecode_str"]}
                              )
      )

      # Non fixed effects model
      models.append( smf.ols(formula_2 + control,
                        data= reg_data
                        ).fit(cov_type='cluster', 
                              cov_kwds={'groups': reg_data["villagecode_str"]}
                              )
      )
      
      # fixed effects: With dummies i.e least sq dummy variable model
      models.append( smf.ols(formula_1 + control + "+ C(tehsil_e)",
                        data= reg_data
                        ).fit(cov_type='cluster', 
                              cov_kwds={'groups': reg_data["villagecode_str"]}
                              )
      )

      # fixed effects: With dummies i.e least sq dummy variable model
      models.append( smf.ols(formula_2 + control + "+ C(tehsil_e)",
                        data= reg_data
                        ).fit(cov_type='cluster', 
                              cov_kwds={'groups': reg_data["villagecode_str"]}
                              )
      )


      ## demeaned fixed effects model (see end for function) ##
      
      # models.append(  fixed_effects_model(  eq = formula_1 + control,
      #                                     data= reg_data,
      #                                     group = "tehsil_e",
      #                                     cluster = "villagecode_str")
      # )

      
      # models.append( fixed_effects_model(  eq = formula_2 + control,
      #                                     data= reg_data,
      #                                     group = "tehsil_e",
      #                                     cluster = "villagecode_str")
      # )

In [9]:
# oreder in which to display the regression coefficients
treat = ["treatment012","treatment_h","treatment_hs"] 
inter_overall = [i+":treatment012" for i in DD_side_char] 
inter_h = [i+":treatment_h" for i in DD_side_char] 
inter_hs = [i+":treatment_hs" for i in DD_side_char] 

reg_order = treat + DD_side_char + inter_overall + inter_h + inter_hs + ["totrefills_omc_b2"]

In [10]:
# Regression table

info_dict={'No. observations' : lambda x: f"{int(x.nobs):d}"}
           
results_table = summary_col(results = models,
                            float_format='%0.3f',
                            stars = True,
                            model_names=['HH Edu Non FE (1)',
                                         'HH Edu Non FE (2)',
                                         'HH Edu FE (1)',
                                         'HH Edu FE (1)',
                                         'HH Wealth Non FE (1)',
                                         'HH Wealth Non FE (2)',
                                         'HH Wealth FE (1)',
                                         'HH Wealth FE (2)',
                                         'PC Decision Non FE (1)',
                                         'PC Decision Non FE (2)',
                                         'PC Decision FE (1)',
                                         'PC Decision FE (2)',
                                         ],
                            info_dict = info_dict,
                            regressor_order= reg_order,
                            drop_omitted=  True
                            )

In [11]:
results_table

,HH Edu Non FE (1) I,HH Edu Non FE (2) I,HH Edu FE (1) I,HH Edu FE (1) II,HH Wealth Non FE (1) I,HH Wealth Non FE (2) I,HH Wealth FE (1) I,HH Wealth FE (2) I,PC Decision Non FE (1) I,PC Decision Non FE (2) I,PC Decision FE (1) I,PC Decision FE (2) I
treatment012,0.198,,0.231*,,0.222,,0.249,,0.067,,0.094,
,(0.125),,(0.127),,(0.220),,(0.222),,(0.101),,(0.098),
treatment_h,,0.070,,0.081,,0.208,,0.216,,-0.005,,0.002
,,(0.136),,(0.143),,(0.257),,(0.264),,(0.118),,(0.120)
treatment_hs,,0.317**,,0.372**,,0.225,,0.264,,0.145,,0.196*
,,(0.156),,(0.153),,(0.244),,(0.242),,(0.122),,(0.119)
edu_hhhead_b,0.183,0.181,0.167,0.164,-0.020,-0.016,-0.045,-0.041,-0.005,0.004,-0.034,-0.026
,(0.125),(0.125),(0.125),(0.125),(0.090),(0.089),(0.091),(0.090),(0.091),(0.091),(0.093),(0.093)
assets_index,0.079,0.082,0.091,0.094,0.142,0.143,0.154,0.152,0.045,0.054,0.056,0.065
,(0.068),(0.068),(0.068),(0.068),(0.110),(0.110),(0.111),(0.111),(0.072),(0.072),(0.072),(0.072)


In [12]:
# function for fixed effects model: group demean
# writing seperate function so that regression is estimated using 
# statsmodels. Linearmodels lib not compatible with summary_col
# function uses group demeaned values to estimate the equation
# another way is to add dummies


def fixed_effects_model(eq = None,data= None, 
                        group = None, cluster = None):
    '''
    date: 17-10-21
    Uses smf.ols, implements group demean 
    Outputs the fitted fixed effects model. Just need to call summary()
    eq: (string) formula statsmodel/R style. Interaction compatable
        Dummy incompatable
    data: Pandas dataframe containing the exog and endog columns
    group: (string) the column name to apply fixed effects on
    cluster: (string) the column name to cluster se on. If same as group,
            specify again
    '''
    # split the formula into column names  
    first_split = [i.replace(" ","") for i in eq.split("~")]
    second_split = first_split[1].split("+")

    # to create interaction columns
    for i in second_split:
        if "*" in i:
                two_cols = i.split("*")
                data[i] = data[two_cols[0]]*data[two_cols[1]]            

    # then demean the columns along with interaction
    mean_val = data.groupby(group).mean()
    demeaned_val = (data.set_index(group) - mean_val)
    
    # run regression
    FE =  smf.ols( eq, 
                   data = demeaned_val
                  ).fit(cov_type='cluster', 
                        cov_kwds={'groups': data[cluster]}
                     )
    
    return FE

## Example using linearmodels library

In [13]:
# all_controls = df[["villagecode_str",   # for clustering
#                    "hhnum_b", "edu_hhhead_b", "occu_b", "age_pc_b", "pc_edu_b", 
#                    "hindu_b", "hh_caste_b", "assets_index",   #  HH controls
#                    "education_b", "healthstatus_b", "road_b", "irrigation_b",  
#                    "subdistricthqdist_c", #  Village controls
#                     # "noncompliance_village",
#                 #    "pcdecisionindex_std_b",
#                    "totrefills_omc_b2", "treatment012", "treatment_h", "treatment_hs",
#                    "totrefills_omc_e2", 
#                    "tehsil_e"      # fixed  effects
#                    ]]  

# all_controls["edu_hhhead_b*treatment012"] = all_controls.edu_hhhead_b * all_controls.treatment012

# form = "totrefills_omc_e2 ~ treatment012 + totrefills_omc_b2 +  edu_hhhead_b*treatment012 " 

# mod = PanelOLS.from_formula(form + control,
#                             data = all_controls.set_index(["tehsil_e",all_controls.index]))

# result = mod.fit(cov_type='clustered', 
#                         cov_kwds={'groups': df.villagecode_str})
# result.summary